In [1]:
library(tidyverse)
library(RPostgreSQL)

Loading tidyverse: ggplot2
Loading tidyverse: tibble
Loading tidyverse: tidyr
Loading tidyverse: readr
Loading tidyverse: purrr
Loading tidyverse: dplyr
Conflicts with tidy packages ---------------------------------------------------
filter(): dplyr, stats
lag():    dplyr, stats
Loading required package: DBI


In [2]:
sql_dir <- file.path("..", "SQL")
list.files(sql_dir)

[1] "cohort.sql"

In [3]:
drv <- dbDriver("PostgreSQL")
con <- dbConnect(drv, dbname = "mimic", user = "lcp_db_user")
dbSendQuery(con, "set search_path=echo,public,mimiciii;")

<PostgreSQLResult>

#### Cohort size (adults in their 1st ICU stay)

In [4]:
dbGetQuery(con, "select count(*) from cohort;")

count
53328


#### Angus

In [5]:
sql <- "
select (select count(*) from cohort) as cohort,
       count(*) as angus
  from cohort
 where angus = 1;
"
dbGetQuery(con, sql) %>% mutate(ratio = angus / cohort * 100)

cohort,angus,ratio
53328,16121,30.2299


#### Echo

In [6]:
sql <- "
select (select count(*) from cohort) as cohort,
       count(*) as echo
  from cohort
 where echo_time is not null;
"
dbGetQuery(con, sql) %>% mutate(ratio = echo / cohort * 100)

cohort,echo,ratio
53328,25967,48.69299


In [7]:
dbDisconnect(con)
dbUnloadDriver(drv)

[1] TRUE

[1] TRUE